In [1]:
import geopandas as gpd
import pandas as pd
import networkx as nx
import osmnx as ox
import folium
import pickle
from shapely.geometry import box, Point

### Carregando o Grafo

In [2]:
# Ler o grafo de um arquivo usando pickle
with open('grafo_linhas.grafo', 'rb') as f:
    G = pickle.load(f)

In [5]:
print(len(G.nodes))

4544


In [4]:
for node, data in G.nodes(data=True):
    print(f"Node  data: {data}")
    break

for _, _, data in G.edges(data=True):
    print(f"Edge data: {data}")
    break


Node  data: {'coords': <POINT (-47.785 -15.763)>}
Edge data: {'dist': 0.0036503507675291267, 'linha': 141.8}


### Receber 2 pontos (origem e destino) com suas respectivas coordenadas 

In [3]:
origem = (-15.989444964529529, -48.044418962814866) # Unb Gama
destino = (-15.79113644987054, -47.88317800514907)   # Conjunto Nacional

G.graph["crs"] = 'EPSG:31983'

def nearest_node(G, long, lat):
    distancias = {}
    ponto = Point(long, lat)
    for node in G.nodes():
        node_coords = Point(G.nodes[node]['coords'])  
        dist = node_coords.distance(ponto)
        distancias[node] = {'dist': dist}
    # Find the nodes with the shortest distancias to the origin and destination
    nearest_node = min(distancias, key=lambda x: distancias[x]['dist'])
    return nearest_node

orig_node = nearest_node(G, origem[1], origem[0])
dest_node = nearest_node(G, destino[1], destino[0])

print(orig_node, dest_node)

7672 3482


### Verificando com o Folium

In [4]:
# # Inicializar o mapa do Folium (ajuste as coordenadas iniciais conforme necessário)
m = folium.Map(location=[-15.7942, -47.8825], zoom_start=12)

# Adicionar as paradas como marcadores no mapa

lat1, long1 = G.nodes[orig_node]['coords'].y, G.nodes[orig_node]['coords'].x
lat2, long2 = G.nodes[dest_node]['coords'].y, G.nodes[dest_node]['coords'].x

folium.Marker([lat1, long1], popup=f'Parada {orig_node}').add_to(m)
folium.Marker([lat2, long2], popup=f'Parada {dest_node}').add_to(m)

m